In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import warnings
from scipy.optimize import dual_annealing,minimize,minimize_scalar
from datetime import datetime
np.set_printoptions(precision=5)

## global Variables
alpha = 1/5.8
gamma = 1/5
epsilon = 0.66
num_units = 38
## sorts sero data,population data and removes rows not present in time series data
## the order of districts after this is same in all the tables
sero1 = pd.read_csv('seroprevalence2.csv')
sero1 = sero1.iloc[:-1,:]
sero1.drop(sero1.loc[sero1['Unit']=='Bengaluru Urban Conglomerate'].index, inplace=True)
sero1.sort_values('Unit',inplace = True)
#print(sero1['Unit'].values)
#sero1 = sero1.iloc[8:,:]


sero2 = pd.read_csv('seroprevalence-round2.csv')
sero2 = sero2.iloc[:-1,:]
sero2.drop(sero2.loc[sero2['Unit']=='Bengaluru Urban Conglomerate'].index, inplace=True)
sero2.sort_values('Unit',inplace = True)
#sero2 = sero2.iloc[:,:]

tests = pd.read_csv('tests.csv')
population = pd.read_csv('population.csv')
population.sort_values('district',inplace = True)
#print(population['district'].values)
pop = np.array(population.loc[:,'population'].to_list())
#population = population.iloc[8:,:]

vaccination = pd.read_csv('vaccination2.csv')
vaccination.rename( columns={'Unnamed: 0':'Date'}, inplace=True )
end_idx = np.where( vaccination['Date'] == 'Dose_2')[0][0]
vacc = (vaccination.iloc[0:end_idx,2:].to_numpy()).astype(int)


tinf = pd.read_csv('timeseries_TINF.csv')
trec = pd.read_csv('timeseries_TREC.csv')
tdec = pd.read_csv('timeseries_TDEC.csv')
tinf = tinf.iloc[:,:-2]
trec = trec.iloc[:,:-2]
tdec = tdec.iloc[:,:-2]
tinf = (tinf.iloc[:,1:].to_numpy()).astype(int)
trec = (trec.iloc[:,1:].to_numpy()).astype(int)
tdec = (tdec.iloc[:,1:].to_numpy()).astype(int)

bengaluru_urban_total_pop = np.sum(pop[0:8]) + pop[12]
bbmp_inf = np.zeros((np.shape(tinf)[0],8))
bbmp_rec = np.zeros((np.shape(trec)[0],8))
bbmp_dec = np.zeros((np.shape(tdec)[0],8))
bbmp_vacc = np.zeros((np.shape(vacc)[0],8))

for i in range(0,8):
    bbmp_inf[:,i] = (pop[i]/bengaluru_urban_total_pop)* tinf[:,4]
    bbmp_rec[:,i] = (pop[i]/bengaluru_urban_total_pop)* trec[:,4]
    bbmp_dec[:,i] = (pop[i]/bengaluru_urban_total_pop)* tdec[:,4]
    bbmp_vacc[:,i] = (pop[i]/bengaluru_urban_total_pop)* vacc[:,4]

tinf[:,4] = (pop[12]/bengaluru_urban_total_pop)* tinf[:,4]
trec[:,4] = (pop[12]/bengaluru_urban_total_pop)* trec[:,4]
tdec[:,4] = (pop[12]/bengaluru_urban_total_pop)* tdec[:,4]
vacc[:,4] = (pop[12]/bengaluru_urban_total_pop)* vacc[:,4]

tinf = np.append(bbmp_inf,tinf,axis = 1)
trec = np.append(bbmp_rec,trec,axis = 1)
tdec = np.append(bbmp_dec,tdec,axis = 1)
vacc = np.append(bbmp_vacc,vacc,axis = 1)
print(vacc.shape)
## returns S0,E0,I0,R0 from sero data on 11th Oct or on 1st March according to flag


(220, 39)


In [3]:
def initial_conditions(flag):
    
    
    if flag == 1:
        
        I0 = np.array(sero1.loc[:,'% active infection'].to_list())
        I0 = (0.01*np.multiply(pop,I0)).astype(int)
        R0 = np.array(sero1.loc[:,'% IgG against SARS-CoV2'].to_list())
        R0 = (0.01*np.multiply(pop,R0)).astype(int)
        CIR0 = np.array(sero1.loc[:,'CIR'].to_list())
        end_idx = 208
        start_idx = 203  ## index of 11th oct in timeseries
        
    elif flag == 3:
        
        I0 = np.zeros(30)
        R0 = np.array(sero2.loc[:,'%-IgG against SARS-CoV2'].to_list())
        R0 = (0.01*np.multiply(pop,R0)).astype(int)
        CIR0 = np.array(sero2.loc[:,'CIR'].to_list())
        start_idx = 344  ## index of 1st march in timeseries
        end_idx = 349
   
    E0 = np.zeros(np.size(I0))
    
    for idx in range(start_idx,end_idx):
        E0 = E0 + np.mean(tinf[idx-6:idx+1,:],axis = 0) -  np.mean(trec[idx-6:idx+1,:],axis = 0) - np.mean(tdec[idx-6:idx+1,:],axis = 0)
    
    #E0 = 1/7*(np.array(Tinf.iloc[1:,end_idx].to_list()) - np.array(Tinf.iloc[1:,start_idx].to_list()))
    E0 = (0.2*np.multiply(E0,CIR0)).astype(int) ## average of infections from 11th to 15th
    S0 = pop - I0 - E0 - R0
    return S0,E0,I0,R0,CIR0,pop



In [4]:
## returns CIR(t) for all districts
## returns 38*num of days
def get_CIR_t(CIR0,flag):
    
    if flag == 1:
        start_idx = np.where(tests['Date'] == '11-10-2020')[0][0]
        end_idx = np.where(tests['Date'] == '01-11-2020')[0][0]
    elif flag == 2:
        start_idx = np.where(tests['Date'] == '01-11-2020')[0][0]
        end_idx = np.where(tests['Date'] == '28-02-2021')[0][0]
    elif flag == 3:
        start_idx = np.where(tests['Date'] == '28-02-2021')[0][0]
        end_idx = np.where(tests['Date'] == '15-03-2021')[0][0]
    elif flag == 4:
        start_idx = np.where(tests['Date'] == '15-03-2021')[0][0]
        end_idx = np.where(tests['Date'] == '07-04-2021')[0][0]    
    duration = end_idx - start_idx + 1 ## inclusive of both start and end
    T_t = tests['Daily Samples Tested'].values
    T0 =  T_t[start_idx]
    T_t = T_t[start_idx:end_idx+1]
    #T_t = np.diff(T_t,prepend = T0)
    CIR_t = np.zeros((num_units,duration))
    
    for j in range(duration):
        if j<7:
            CIR_t[:,j] = T0/np.mean(T_t[0:j+1])*CIR0
        elif j!=duration-1:
            CIR_t[:,j] = T0/np.mean(T_t[j-6:j+1])*CIR0
        elif j == duration-1:
            CIR_t[:,j] = T0/np.mean(T_t[j-6:j+1])*CIR0
    
    return CIR_t


In [5]:
## returns no of infections according to duration for all districts
# returns 38*num of days
def get_c_t(idx,flag):
    
    if flag == 1:
        start_idx = np.where(tests['Date'] == '11-10-2020')[0][0]
        end_idx = np.where(tests['Date'] == '01-11-2020')[0][0]
    elif flag == 2:
        start_idx = np.where(tests['Date'] == '01-11-2020')[0][0]
        end_idx = np.where(tests['Date'] == '28-02-2021')[0][0]
    elif flag == 3:
        start_idx = np.where(tests['Date'] == '28-02-2021')[0][0]
        end_idx = np.where(tests['Date'] == '15-03-2021')[0][0]
    elif flag == 4:
        start_idx = np.where(tests['Date'] == '15-03-2021')[0][0]
        end_idx = np.where(tests['Date'] == '07-04-2021')[0][0]
           
    duration = end_idx - start_idx + 1   ## inclusive of both start and end
    C_t = 0.5*(tinf[start_idx:end_idx+1,idx] - trec[start_idx:end_idx+1,idx] - tdec[start_idx:end_idx+1,idx])
    #C_t = tinf.iloc[:,1+idx].to_numpy()
    #print(np.shape(C_t))
    #cases_start = C_t[start_idx] - C_t[start_idx-1]
    #C_t = C_t[start_idx:end_idx+1]
    #C_t = np.diff(C_t,prepend = cases_start)
    #C_t = np.transpose(C_t)
    c_t = C_t
    
    for j in range(duration):
        
        if j== 0:
            c_t[j] = C_t[j]
        elif j<7:
            c_t[j] = np.mean(C_t[0:j+1])
        elif j!=duration-1:
            c_t[j] = np.mean(C_t[j-6:j+1]) 
        elif j==duration-1:
            c_t[j] = np.mean(C_t[j-6]) 
    return c_t


In [6]:
## returns vaccinations for a given district
def get_v_t(idx):
    
    #Total_V_t = data_preprocessing()[-3]
    #print(Total_V_t)
    #print(V_t)
    V_t = vacc[:,idx]
    V_t = np.insert(V_t,13,np.mean(V_t[6:13]))
    V_t = np.insert(V_t,14,np.mean(V_t[7:14]))
    V_t = np.insert(V_t,36,np.mean(V_t[30:36]))
    V_t = np.insert(V_t,40,np.mean(V_t[34:40]))
    V_t = np.insert(V_t,41,np.mean(V_t[34:41]))
    '''
    for j in range(np.size(V_t)):
        if j== 0:
            v_t[j] = V_t[j]
        elif j<7:
            v_t[j] = np.mean(V_t[0:j+1])
        elif j!= np.size(V_t)-1:
            v_t[j] = np.mean(V_t[j-6:j+1]) 
        elif j == np.size(V_t)-1:
            v_t[j] = np.mean(V_t[j-6:]) 
    '''
    return V_t


In [7]:
def get_delta_W_t(length,R0):
    
    delta_W_t = 0
    shape = 3.6
    scale = 120
    for i in range(length):
        delta_W_t += (shape/scale)*(((i+1)/scale)**(shape-1))*math.exp(-((i+1)/scale)**shape)
        
    return delta_W_t*R0                                        

In [14]:
## input - S0,E0,I0,R0,CIR0,N for a given district with index idx 

def SEIRV_Model(beta,S0,E0,I0,R0,N,idx,flag):
    
    #beta = beta[0]
    #trec = data_preprocessing()[3]
    #print(beta,S0,E0,I0,R0,N,idx,flag)
    
    if flag == 1:
        
        start = datetime(2020,10,11)
        end = datetime(2020,11,1) 
        duration = int((end-start).total_seconds()/(60*60*24))+1
    
    elif flag == 2:
        
        start_simulation = datetime(2020,11,1)
        end_simulation = datetime(2021,2,28)
        vacc_start = datetime(2021,1,18)
        start_idx = int((vacc_start - start_simulation).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
        V_t = get_v_t(idx)
        #print(V_t)
        
    elif flag == 3:
        
        start_simulation = datetime(2021,2,28)
        end_simulation = datetime(2021,3,15)
        vacc_start = datetime(2021,1,18)
        start_idx = int(( start_simulation - vacc_start).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
        V_t = get_v_t(idx)
    
    elif flag == 4:
        
        start_simulation = datetime(2021,3,15)
        end_simulation = datetime(2021,4,7)
        vacc_start = datetime(2021,1,18)
        start_idx = int(( start_simulation - vacc_start).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
        V_t = get_v_t(idx)
        
    S_t = np.zeros(duration)
    S_t[0] = S0
    E_t = np.zeros(duration)
    E_t[0] = E0
    I_t = np.zeros(duration)
    I_t[0] = I0
    R_t = np.zeros(duration)
    R_t[0] = R0
    #st = np.where(trec['Date'] == '01-11-2020')[0][0]
    delta_V_t = 0 
    delta_W_t = 0 
    
    for i in range(0,duration-1):
        
        '''
        if flag == 1:
            delta_W_t = trec[i+203 - 109,idx] - trec[i+203 - 110,idx]
        ''' 
        
        if flag == 2:
            if i < start_idx:
                delta_V_t = 0
            else :
                delta_V_t = V_t[i-start_idx]
            
        elif flag == 3 or flag == 4:
            
            delta_V_t = V_t[i + start_idx ] 
            
        if i >0:
            delta_W_t = get_delta_W_t(duration-i,R_t[i-1])
            
           
        '''       
        if i<110:
            delta_W_t = trec.iloc[i + st - 109][1+idx] - trec.iloc[i + st-110][1+idx]
        else:
            delta_W_t = R_t[i-109] - R_t[i-110]
        )
        '''
        #print( S_t[i],  beta,I_t[i]/N,epsilon*delta_V_t )
        S_t[i+1] = S_t[i] - beta*S_t[i]*(I_t[i]/N) - epsilon*delta_V_t + delta_W_t
        E_t[i+1] = E_t[i] + beta*S_t[i]*(I_t[i]/N) - alpha*E_t[i]
        I_t[i+1] = I_t[i] + alpha*E_t[i] - gamma*I_t[i]
        R_t[i+1] = R_t[i] + gamma*I_t[i] +  epsilon*delta_V_t  - delta_W_t
    
    return S_t,E_t,I_t,R_t


In [15]:
## input - S0,E0,I0,R0,CIR0,N for a given district with index idx 
def evolution(beta,S_prev,E_prev,I_prev,R_prev,N,delta_W_t,delta_V_t,flag):
    
    S_next = np.zeros(num_units)
    E_next = np.zeros(num_units)
    I_next = np.zeros(num_units)
    R_next = np.zeros(num_units)
    R_day = np.zeros(num_units)
    
    if flag!=4:
        Mobi_Mat = np.identity(num_units)
    else:
        Mobi_Mat = 0.99*np.identity(num_units) + 0.01*np.ones((num_units,num_units))
    #delta_V_t = np.zeros(num_units)
    
    for i in range(num_units):
        
        su_mobi = 0
        
        for j in range(num_units):
                
            su_mobi += Mobi_Mat[i,j]*I_prev[j]/N[j]  
                
                
        S_next[i] = S_prev[i] - beta[i]*S_prev[i]*(su_mobi) - epsilon*delta_V_t[i] + delta_W_t[i] 
        E_next[i] = E_prev[i] + beta[i]*S_prev[i]*(su_mobi) - alpha*E_prev[i] 
        I_next[i] = I_prev[i] + alpha*E_prev[i] - gamma*I_prev[i]
        R_next[i] = R_prev[i] + gamma*I_prev[i] +  epsilon*delta_V_t[i] - delta_W_t[i]   
        R_day[i] = gamma*I_prev[i] +  epsilon*delta_V_t[i] - delta_W_t[i]
    
    return S_next,E_next,I_next,R_next,R_day

def SEIRV_Model2(beta,S_prev,E_prev,I_prev,R_prev,N,flag):
    
    #beta = beta[0]
    #trec = data_preprocessing()[3]
    #print(beta,S0,E0,I0,R0,N,idx,flag)
    
    if flag == 1:
        
        start = datetime(2020,10,11)
        end = datetime(2020,11,1) 
        duration = int((end-start).total_seconds()/(60*60*24))+1
    
    elif flag == 2:
        
        start_simulation = datetime(2020,11,1)
        end_simulation = datetime(2021,2,28)
        vacc_start = datetime(2021,1,18)
        start_idx = int((vacc_start - start_simulation).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
        #print(V_t)
        
    elif flag == 3:
        
        start_simulation = datetime(2021,2,28)
        end_simulation = datetime(2021,3,15)
        vacc_start = datetime(2021,1,18)
        start_idx = int(( start_simulation - vacc_start).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
        
    elif flag == 4:
        
        start_simulation = datetime(2021,3,15)
        end_simulation = datetime(2021,4,7)
        vacc_start = datetime(2021,1,18)
        start_idx = int(( start_simulation - vacc_start).total_seconds()/(60*60*24))
        duration = int((end_simulation - start_simulation).total_seconds()/(60*60*24))+1
    #st = np.where(trec['Date'] == '01-11-2020')[0][0]
    delta_V_t = np.zeros(num_units) 
    delta_W_t = np.zeros(num_units) 
    S = np.zeros((num_units,duration))
    E = np.zeros((num_units,duration))
    I = np.zeros((num_units,duration))
    R = np.zeros((num_units,duration))
    R_day = np.zeros((num_units,duration))
    S[:,0] =  S_prev
    E[:,0] =  E_prev
    I[:,0] =  I_prev
    R[:,0] =  R_prev
    R_day[:,0] = R_prev
    for i in range(0,duration-1):
        
        '''
        if flag == 1:
            delta_W_t = trec[i+203 - 109,idx] - trec[i+203 - 110,idx]
        ''' 
        
        if flag == 2:
            if i < start_idx:
                delta_V_t = np.zeros(num_units)
            else :
                delta_V_t = vacc[i-start_idx,:]
                '''
                for j in range(num_units):
                    delta_V_t[j] = get_v_t(j)[i-start_idx] - get_v_t(j)[i-start_idx-1]
                '''
            #print(delta_V_t)
        elif flag == 3 or flag == 4:
            delta_V_t = vacc[i+start_idx,:]
            '''
            for j in range(num_units):
                delta_V_t[j] = get_v_t(j)[i + start_idx] - get_v_t(j)[i + start_idx-1] 
            '''
        if i >0:
            
            delta_W_t = get_delta_W_t(duration-i,R[:,i-1])
            
            
        
        S_prev,E_prev,I_prev,R_prev,R_day_prev = evolution(beta,S_prev,E_prev,I_prev,R_prev,N,delta_W_t,delta_V_t,flag)
        S[:,i+1] =  S_prev
        E[:,i+1] =  E_prev
        I[:,i+1] =  I_prev
        
        R[:,i+1] =  R_prev
        R_day[:,i+1] = R_day_prev
        '''       
        if i<110:
            delta_W_t = trec.iloc[i + st - 109][1+idx] - trec.iloc[i + st-110][1+idx]
        else:
            delta_W_t = R_t[i-109] - R_t[i-110]
            #print(delta_V_t)
        '''
        
    return S,E,I,R
    

In [24]:
## idx - index of district
def loss_function(beta,S0,E0,I0,R0,CIR0,N,idx,flag):
    
    #print(beta,S0,E0,I0,R0,N,idx,flag)
    if flag == 1:
        start = datetime(2020,10,11)
        end = datetime(2020,11,1)
    elif flag == 2:
        start = datetime(2020,11,1)
        end = datetime(2021,2,28)
    elif flag == 3:
        start = datetime(2021,2,28)
        end = datetime(2021,3,15)
    elif flag == 4:
        start = datetime(2021,3,15)
        end = datetime(2021,4,7)
    CIR_t = get_CIR_t(CIR0,flag)[idx,:] # on 1st Novemeber
    c_t   = get_c_t(idx,flag)
    duration = int((end-start).total_seconds()/(60*60*24))+1 ## inclusive of start and end
    E_t   = SEIRV_Model(beta,S0,E0,I0,R0,N,idx,flag)[1]
    '''
    if flag == 1 or flag ==2:
        
        E_t   = SEIRV_Model(beta,S0,E0,I0,R0,N,idx,flag)[1]
        
    elif flag == 3:
        
        E_t   = SEIRV_Model2(beta,S0,E0,I0,R0,N,flag)[1]
    '''
    if flag == 1 or flag == 3:
        
        e_t = E_t[-1]/CIR_t[-1] # on 1st November
        cases_1stNov = c_t[-1]
        loss  = abs(alpha*e_t/cases_1stNov-1)
        #print(idx,alpha*e_t,cases_1stNov,loss)
        
    else:
        e_t = E_t/CIR_t # on 1st November
        mean_e_t = np.zeros(duration) ## mean of delta i(t) from t-6 to t day
        for i in range(duration):
            if i<7:
                mean_e_t[i] = max(np.mean(e_t[0:i+1]),1/alpha)
            elif i!= duration-1:
                mean_e_t[i] = max(np.mean(e_t[i-6:i+1]),1/alpha)
            elif i == duration-1:
                mean_e_t[i] = max(np.mean(e_t[i-6:]),1/alpha)

        loss  = np.sum(np.square(np.log(alpha*mean_e_t) - np.log(c_t)))/duration
        
    return loss


In [25]:
def loss_function2(beta,S0,E0,I0,R0,CIR0,N,flag):
    
    #print(beta,S0,E0,I0,R0,N,idx,flag)
    start = datetime(2021,3,15)
    end = datetime(2021,4,7)
    CIR_t = get_CIR_t(CIR0,flag) # on 1st Novemeber
    
    duration = int((end-start).total_seconds()/(60*60*24))+1 ## inclusive of start and end
    E_t   = SEIRV_Model2(beta,S0,E0,I0,R0,N,flag)[1]
    
    loss = 0
    mean_e_t = np.zeros(duration) ## mean of delta i(t) from t-6 to t day
    for i in range(num_units):
        e_t = np.divide(E_t[i,:],CIR_t[i,:])
        c_t   = get_c_t(i,flag)   
        for j in range(duration):
            if j<7:
                mean_e_t[j] = max(np.mean(e_t[0:j+1]),1/alpha)
            elif j!= duration-1:
                mean_e_t[j] = max(np.mean(e_t[j-6:j+1]),1/alpha)
            elif j == duration-1:
                mean_e_t[j] = max(np.mean(e_t[j-6:]),1/alpha)
        loss += np.sum(np.square(np.log(alpha*mean_e_t) - np.log(c_t)))/duration
        
    return loss

    

In [26]:
## since mobility is identity so minimized independently over all districts
def minimizer(S0,E0,I0,R0,CIR0,N,flag):
    
    num_units = np.size(S0)
    beta = np.zeros(num_units)
    #if flag == 1 or flag == 2:
    if flag!=4: 
        print('losses for phase %d'%flag)
        for i in range(num_units):
            sol = minimize(loss_function,x0 = 0.5,args = (S0[i],E0[i],I0[i],R0[i],CIR0,N[i],i,flag),bounds = [[0,1]])
            beta[i] = sol.x
            print('loss for %d district is %f'%(i,sol.fun))
        
    else :
        print('losses for phase %d'%flag)
        beta_init = [0.5 for i in range(0,38)]
        bnds = [[0,1] for i in range(0,38)]
        sol = minimize(loss_function2,x0 = beta_init,args = (S0,E0,I0,R0,CIR0,N,flag),bounds = bnds)
        beta = sol.x
        print('Overall loss for all districts is %f'%(sol.fun))
    return beta  

'''
for i in range(38):
    #cases = tinf.to_numpy()[start_idx:end_idx+1,1:] - trec.to_numpy()[start_idx:end_idx+1,1:] - tdec.to_numpy()[start_idx:end_idx+1,1:]
    y = get_c_t(i,2)
    x = np.arange(1,len(y)+1)
    plt.figure()
    plt.plot(x,y)
    plt.show()
    plt.close()
'''    


'\nfor i in range(38):\n    #cases = tinf.to_numpy()[start_idx:end_idx+1,1:] - trec.to_numpy()[start_idx:end_idx+1,1:] - tdec.to_numpy()[start_idx:end_idx+1,1:]\n    y = get_c_t(i,2)\n    x = np.arange(1,len(y)+1)\n    plt.figure()\n    plt.plot(x,y)\n    plt.show()\n    plt.close()\n'

In [27]:
S0,E0,I0,R0,CIR0,N = initial_conditions(1)
#print(E0)
#print(S0,E0,I0,R0)
beta = minimizer(S0,E0,I0,R0,CIR0,N,1)
print('beta for first phase is',end=' ')
print(beta)

'''
beta  = np.array([0.006, 0.01 , 0.013, 0.019,0.018, 0.019, 0.02, 0.014, 0.014, 0.021,
 0.014, 0.014, 0.01, 0.007, 0.017, 0.01, 0.008, 0.021, 0.017,
 0.015, 0.023, 0.017, 0.01, 0.011, 0.009, 0.017, 0.011, 0.012,
 0.025, 0.007])
'''
CIR0 = get_CIR_t(CIR0,1)[:,-1]
## Code for prediction on 1st November obtaining new S0,E0,I0,R0,CIR0 for next phase 
for i in range(num_units):
    S,E,I,R  = SEIRV_Model(beta[i],S0[i],E0[i],I0[i],R0[i],N[i],i,1)
    actual = get_c_t(i,1)[-1]
    S0[i] = max(S[-1],0) ## S at 1st November
    E0[i] = max(E[-1],0)
    I0[i] = max(I[-1],0)
    R0[i] = max(R[-1],0)
    predicted = alpha*E0[i]/CIR0[i]
    print( 'predicted cases on 1st November for district %d = %f and actual is %f'%(i,predicted,actual))
    
## minimization from 1st to 28th February
beta = minimizer(S0,E0,I0,R0,CIR0,N,2)
print('beta for second phase is',end=' ')
print(beta)
CIR0 = get_CIR_t(CIR0,2)[:,-1]
## Code for prediction on 1st March obtaining new S0,E0,I0,R0,CIR0 for next phase 
for i in range(np.size(S0)):
    
    S,E,I,R  = SEIRV_Model(beta[i],S0[i],E0[i],I0[i],R0[i],N[i],i,2)
    actual = get_c_t(i,2)[-1]
    S0[i] = max(S[-1],0) ## S at 1st November
    E0[i] = max(E[-1],0)
    I0[i] = max(I[-1],0)
    R0[i] = max(R[-1],0)
    predicted = alpha*E0[i]/CIR0[i]
    print('predicted cases on 28th Feb 2021 for district %d = %f and actual is %f'%(i,predicted,actual))

## minimization from 1st march to 15th march

beta = minimizer(S0,E0,I0,R0,CIR0,N,3)
print('beta for third phase is',end=' ')
print(beta)
CIR0 = get_CIR_t(CIR0,3)[:,-1]
for i in range(np.size(S0)):
    S,E,I,R  = SEIRV_Model(beta[i],S0[i],E0[i],I0[i],R0[i],N[i],i,3)
    actual = get_c_t(i,3)[-1]
    S0[i] = max(S[-1],0) ## S at 1st November
    E0[i] = max(E[-1],0)
    I0[i] = max(I[-1],0)
    R0[i] = max(R[-1],0)
    predicted = alpha*E0[i]/CIR0[i]
    print( 'predicted cases on 15th march for district %d = %f and actual is %f'%(i,predicted,actual))

beta = minimizer(S0,E0,I0,R0,CIR0,N,4)
print('beta for fourth phase is',end=' ')
print(beta)
CIR0 = get_CIR_t(CIR0,4)[:,-1]

S,E,I,R  = SEIRV_Model2(beta,S0,E0,I0,R0,N,4)
S0 = S[:,-1] ## S at 1st November
E0 = E[:,-1]
I0 = I[:,-1]
R0 = R[:,-1]
predicted = alpha*E0/CIR0
for i in range(num_units):
    actual = get_c_t(i,4)[-1]
    print( 'predicted cases on 7th April for district %d = %f and actual is %f'%(i,predicted[i],actual))

losses for phase 1
loss for 0 district is 0.609403
loss for 1 district is 0.049674
loss for 2 district is 0.413286
loss for 3 district is 0.000000
loss for 4 district is 0.660152
loss for 5 district is 0.844421
loss for 6 district is 0.787610
loss for 7 district is 0.000000
loss for 8 district is 0.000000
loss for 9 district is 0.163225
loss for 10 district is 0.000000
loss for 11 district is 0.781486
loss for 12 district is 0.083475
loss for 13 district is 0.000000
loss for 14 district is 0.000000
loss for 15 district is 0.000000
loss for 16 district is 0.156747
loss for 17 district is 0.067101
loss for 18 district is 0.000000
loss for 19 district is 0.299367
loss for 20 district is 0.000000
loss for 21 district is 0.000000
loss for 22 district is 0.391953
loss for 23 district is 0.000000
loss for 24 district is 0.000000
loss for 25 district is 0.000000
loss for 26 district is 0.000000
loss for 27 district is 0.000000
loss for 28 district is 0.000000
loss for 29 district is 0.000000
l